# Greenplum Database  Concepts Explained - Part 3

This is Part 3 of Greenplum Database  Concepts Explained, ***MPP Fundamentals and Partitioning***. 

- If you missed Part 1 (*Setup, Describe Input Dataset & Data Loading*) or wish to repeat, then click [here](AWS-GP-demo-1.ipynb).
- If you missed Part 2 (*Basic Table Functions*) or wish to repeat, then click [here](AWS-GP-demo-2.ipynb).

In [1]:
import os, re
from IPython.display import display_html

import pygments.lexers
from pygments import highlight
from pygments.formatters import HtmlFormatter

CONNECTION_STRING = os.getenv('AWSGPDBCONN')

cs = re.match('^postgresql:\/\/(\S+):(\S+)@(\S+):(\S+)\/(\S+)$', CONNECTION_STRING)

DB_USER   = cs.group(1)
DB_PWD    = cs.group(2)
DB_SERVER = cs.group(3)
DB_PORT   = cs.group(4)
DB_NAME   = cs.group(5)

%reload_ext sql
%sql $CONNECTION_STRING

In [2]:
query = "SHOW gp_autostats_mode; \
ALTER DATABASE {} SET gp_autostats_mode TO 'NONE'; \
SHOW gp_autostats_mode;".format(DB_NAME)

%sql $DB_USER@$DB_NAME {''.join(query)}

1 rows affected.
Done.
1 rows affected.


gp_autostats_mode
none


## 5. MPP Fundamentals

This topic provides an overview of how a MPP (*Massively Parallel Processing*) database such as Greenplum, processes queries. Understanding this process can be useful when writing and tuning queries.

Users issue queries to Greenplum Database as they would to any database management system. They connect to the database instance on the Greenplum master host using a client application such as `psql` and submit SQL statements.

### 5.1. Understanding Parallel Query Execution & Greenplum Query Optimizer

Greenplum creates a number of database processes to handle the work of a query. On the master, the query worker process is called the **query dispatcher** (QD). The QD is responsible for creating and dispatching the query plan. It also accumulates and presents the final results. On the segments, a query worker process is called a **query executor** (QE). A QE is responsible for completing its portion of work and communicating its intermediate results to the other worker processes.

There is at least one worker process assigned to each slice of the query plan. A worker process works on its assigned portion of the query plan independently. During query execution, each segment will have a number of processes working on the query in parallel.

The Greenplum Database cost-based optimizer evaluates many strategies for executing a query and chooses the least costly method. The optimizer produces plans based on statistics generated for tables. It is important to have accurate statistics to produce the best plan. The Greenplum optimizer takes into account factors such as,
- the number of rows in tables to be joined, 
- availability of indexes, and 
- cardinality of column data when calculating the costs of alternative execution plans. 

The optimizer also accounts for the location of the data, **preferring to perform as much of the work as possible on the segments and to minimize the amount of data that must be transmitted between segments to complete the query**.

Check the following example:

### Example 1: Forcing data into a single segment

In [3]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/5-1-amzn-reviews-by-marketplace.sql
display_html('\n'.join(sqlfilecode), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 DROP TABLE IF EXISTS demo . amzn_reviews_by_marketplace ; 
 
 CREATE TABLE demo . amzn_reviews_by_marketplace ( 
 marketplace text NULL , 
 customer_id int8 NULL , 
 review_id text NULL , 
 product_id text NULL , 
 product_parent int8 NULL , 
 product_title text NULL , 
 product_category text NULL , 
 star_rating int4 NULL , 
 helpful_votes int4 NULL , 
 total_votes int4 NULL , 
 vine text NULL , 
 verified_purchase text NULL , 
 review_headline text NULL , 
 review_body text NULL , 
 review_date date NULL 
 ) 
 DISTRIBUTED BY ( marketplace ); 
 
 INSERT INTO demo . amzn_reviews_by_marketplace 
 SELECT * FROM demo . amzn_reviews ;

In [4]:
query = !cat script/5-1-amzn-reviews-by-marketplace.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

Done.
Done.
0 rows affected.


[]

#### Let's compare the two tables (*demo.amzn_reviews*, *demo.amzn_reviews_by_marketplace*) data distribution across segments:

In [5]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l psql script/5-1-data-distribution-two-tables.sql
display_html('\n'.join(sqlfilecode), raw=True)

query = !cat script/5-1-data-distribution-two-tables.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT * 
 FROM ( SELECT gp_segment_id :: INT AS "Segment ID" , 
 count ( * ) :: INT AS "Row Count" , 
 'demo.amzn_reviews' AS "Table Name" 
 FROM 
 demo . amzn_reviews 
 GROUP BY 1 ) A 
 UNION ALL 
 SELECT * 
 FROM ( 
 SELECT gp_segment_id :: INT AS "Segment ID" , 
 count ( * ) :: INT AS "Row Count" , 
 'demo.amzn_reviews_by_marketplace' AS "Table Name" 
 FROM 
 demo . amzn_reviews_by_marketplace 
 GROUP BY 1 ) B 
 ORDER BY 1 , 3

49 rows affected.


Segment ID,Row Count,Table Name
0,3143852,demo.amzn_reviews
1,3149928,demo.amzn_reviews
2,3147633,demo.amzn_reviews
3,3143518,demo.amzn_reviews
4,3146928,demo.amzn_reviews
5,3143884,demo.amzn_reviews
6,3144574,demo.amzn_reviews
6,150955707,demo.amzn_reviews_by_marketplace
7,3143894,demo.amzn_reviews
8,3143562,demo.amzn_reviews


### Let's compare the `EXPLAIN ANALYSE` output for the two tables (*demo.amzn_reviews*, *demo.amzn_reviews_by_marketplace*)

In [6]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l psql script/5-2-explain-analyze-count-star-1.sql
display_html('\n'.join(sqlfilecode), raw=True)

explain_output = !cat script/5-2-explain-analyze-count-star-1.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 
display_html('\n'.join(explain_output), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT COUNT ( * ) 
 FROM demo . amzn_reviews ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 QUERY PLAN 
 -------------------------------------------------------------------------------------------------------------------------------------------- 
 Aggregate ( cost = 0.00..431.00 rows = 1 width = 8 ) ( actual time = 29214.104..29214.104 rows = 1 loops = 1 ) 
 -> Gather Motion 48 : 1 ( slice1 ; segments : 48 ) ( cost = 0.00..431.00 rows = 1 width = 1 ) ( actual time = 0.839..18591.963 rows = 150955707 loops = 1 ) 
 -> Seq Scan on amzn_reviews ( cost = 0.00..431.00 rows = 1 width = 1 ) ( actual time = 0.030..660.576 rows = 3150425 loops = 1 ) 
 Planning time : 1.839 ms 
 ( slice0 ) Executor memory : 567 K bytes . 
 ( slice1 ) Executor memory : 41 K bytes avg x 48 workers , 41 K bytes max ( seg0 ) . 
 Memory used : 3223552 kB 
 Optimizer : Pivotal Optimizer ( GPORCA ) 
 Execution time : 29214.949 ms 
 ( 9 rows )

In [7]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l psql script/5-2-explain-analyze-count-star-2.sql
display_html('\n'.join(sqlfilecode), raw=True)

explain_output = !cat script/5-2-explain-analyze-count-star-2.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 
display_html('\n'.join(explain_output), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT COUNT ( * ) 
 FROM demo . amzn_reviews_by_marketplace ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 QUERY PLAN 
 ----------------------------------------------------------------------------------------------------------------------------------------------- 
 Aggregate ( cost = 0.00..431.00 rows = 1 width = 8 ) ( actual time = 43006.288..43006.288 rows = 1 loops = 1 ) 
 -> Gather Motion 48 : 1 ( slice1 ; segments : 48 ) ( cost = 0.00..431.00 rows = 1 width = 1 ) ( actual time = 0.736..32452.489 rows = 150955707 loops = 1 ) 
 -> Seq Scan on amzn_reviews_by_marketplace ( cost = 0.00..431.00 rows = 1 width = 1 ) ( actual time = 0.027..32326.410 rows = 150955707 loops = 1 ) 
 Planning time : 7.213 ms 
 ( slice0 ) Executor memory : 573 K bytes . 
 ( slice1 ) Executor memory : 26 K bytes avg x 48 workers , 41 K bytes max ( seg6 ) . 
 Memory used : 3223552 kB 
 Optimizer : Pivotal Optimizer ( GPORCA ) 
 Execution time : 43007.088 ms 
 ( 9 rows )

### Let's run `ANALYZE <tablename>` to collect table statistics, and compare again the `EXPLAIN ANALYSE` output for the two tables (*demo.amzn_reviews*, *demo.amzn_reviews_by_marketplace*)

In [8]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l psql script/5-2-explain-analyze-count-star-3.sql
display_html('\n'.join(sqlfilecode), raw=True)

explain_output = !cat script/5-2-explain-analyze-count-star-3.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 
display_html('\n'.join(explain_output), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 ANALYSE demo . amzn_reviews ; 
 
 EXPLAIN ANALYZE 
 SELECT COUNT(*) 
 FROM demo.amzn_reviews;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 ANALYZE 
 QUERY PLAN 
 ----------------------------------------------------------------------------------------------------------------------------------------- 
 Aggregate ( cost = 0.00..1158.70 rows = 1 width = 8 ) ( actual time = 1487.059..1487.059 rows = 1 loops = 1 ) 
 -> Gather Motion 48 : 1 ( slice1 ; segments : 48 ) ( cost = 0.00..1158.70 rows = 1 width = 8 ) ( actual time = 836.535..1487.026 rows = 48 loops = 1 ) 
 -> Aggregate ( cost = 0.00..1158.70 rows = 1 width = 8 ) ( actual time = 853.625..853.625 rows = 1 loops = 1 ) 
 -> Seq Scan on amzn_reviews ( cost = 0.00..1152.85 rows = 3147377 width = 1 ) ( actual time = 0.027..636.120 rows = 3150425 loops = 1 ) 
 Planning time : 6.891 ms 
 ( slice0 ) Executor memory : 67 K bytes . 
 ( slice1 ) Executor memory : 25 K bytes avg x 48 workers , 25 K bytes max ( seg0 ) . 
 Memory used : 3223552 kB 
 Optimizer : Pivotal Optimizer ( GPORCA ) 
 Execution time : 1487.817 ms 
 ( 10 rows )

In [9]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l psql script/5-2-explain-analyze-count-star-4.sql
display_html('\n'.join(sqlfilecode), raw=True)

explain_output = !cat script/5-2-explain-analyze-count-star-4.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 
display_html('\n'.join(explain_output), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 ANALYSE demo . amzn_reviews_by_marketplace ; 
 
 EXPLAIN ANALYZE 
 SELECT COUNT(*) 
 FROM demo.amzn_reviews_by_marketplace;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 ANALYZE 
 QUERY PLAN 
 ------------------------------------------------------------------------------------------------------------------------------------------------------------ 
 Aggregate ( cost = 0.00..1144.33 rows = 1 width = 8 ) ( actual time = 41698.330..41698.331 rows = 1 loops = 1 ) 
 -> Gather Motion 48 : 1 ( slice1 ; segments : 48 ) ( cost = 0.00..1144.33 rows = 1 width = 8 ) ( actual time = 0.004..41698.298 rows = 48 loops = 1 ) 
 -> Aggregate ( cost = 0.00..1144.33 rows = 1 width = 8 ) ( actual time = 0.003..0.003 rows = 1 loops = 1 ) 
 -> Seq Scan on amzn_reviews_by_marketplace ( cost = 0.00..1138.45 rows = 3160367 width = 1 ) ( actual time = 0.024..31575.312 rows = 150955707 loops = 1 ) 
 Planning time : 6.889 ms 
 ( slice0 ) Executor memory : 67 K bytes . 
 ( slice1 ) Executor memory : 25 K bytes avg x 48 workers , 25 K bytes max ( seg0 ) . 
 Memory used : 3223552 kB 
 Optimizer : Pivotal Optimizer ( GPORCA ) 
 Execution time : 41699.202 ms 
 ( 10 rows )

## 6. Partitioning

Table partitioning enables supporting very large tables, such as fact tables, by logically dividing them into smaller, more manageable pieces. Partitioned tables can improve query performance by allowing the Greenplum Database query optimizer to scan only the data needed to satisfy a given query instead of scanning all the contents of a large table.

### 6.1. Create a new copy of the original table, define a *PARTITION* pattern (by month) and load it.

After you create the partitioned table structure, top-level parent tables are empty. Data is routed to the bottom-level child table partitions. In a multi-level partition design, only the subpartitions at the bottom of the hierarchy can contain data.

Rows that cannot be mapped to a child table partition are rejected and the load fails. To avoid unmapped rows being rejected at load time, define your partition hierarchy with a DEFAULT partition. Any rows that do not match a partition's CHECK constraints load into the DEFAULT partition.

At runtime, the query optimizer scans the entire table inheritance hierarchy and uses the CHECK table constraints to determine which of the child table partitions to scan to satisfy the query's conditions. The DEFAULT partition (if your hierarchy has one) is always scanned. DEFAULT partitions that contain data slow down the overall scan time.

When you use COPY or INSERT to load data into a parent table, the data is automatically rerouted to the correct partition, just like a regular table.

In [5]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-1-create-and-load-partition-table.sql
display_html('\n'.join(sqlfilecode), raw=True)

query = !cat script/6-1-create-and-load-partition-table.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 DROP TABLE IF EXISTS demo . amzn_reviews_partitioned ; 

 CREATE TABLE demo . amzn_reviews_partitioned ( 
 marketplace TEXT , 
 customer_id BIGINT , 
 review_id TEXT , 
 product_id TEXT , 
 product_parent BIGINT , 
 product_title TEXT , 
 product_category TEXT , 
 star_rating INTEGER , 
 helpful_votes INTEGER , 
 total_votes INTEGER , 
 vine TEXT , 
 verified_purchase TEXT , 
 review_headline TEXT , 
 review_body TEXT , 
 review_date DATE ) 
 DISTRIBUTED BY ( review_id ) 
 PARTITION BY RANGE ( review_date )( 
 START ( '1995-01-01' :: date ) 
 END ( '2015-12-31' :: date ) INCLUSIVE 
 EVERY ( '1 year' :: interval ) 
 ); 

 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '1995-01-01' ) TO year1995 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '1996-01-01' ) TO year1996 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '1997-01-01' ) TO year1997 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '1998-01-01' ) TO year1998 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '1999-01-01' ) TO year1999 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2000-01-01' ) TO year2000 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2001-01-01' ) TO year2001 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2002-01-01' ) TO year2002 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2003-01-01' ) TO year2003 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2004-01-01' ) TO year2004 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2005-01-01' ) TO year2005 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2006-01-01' ) TO year2006 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2007-01-01' ) TO year2007 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2008-01-01' ) TO year2008 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2009-01-01' ) TO year2009 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2010-01-01' ) TO year2010 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2011-01-01' ) TO year2011 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2012-01-01' ) TO year2012 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2013-01-01' ) TO year2013 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2014-01-01' ) TO year2014 ; 
 ALTER TABLE demo . amzn_reviews_partitioned RENAME PARTITION FOR ( '2015-01-01' ) TO year2015 ; 
 
 INSERT INTO demo . amzn_reviews_partitioned 
 SELECT * FROM demo . amzn_reviews ; 

 ANALYSE demo . amzn_reviews_partitioned ; 
 
 SELECT COUNT ( * ) AS row_count , 'demo.amzn_reviews' tablename FROM demo . amzn_reviews 
 UNION ALL 
 SELECT COUNT ( * ) AS row_count , 'demo.amzn_reviews_partitioned' tablename FROM demo . amzn_reviews_partitioned ;

Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
0 rows affected.
Done.
2 rows affected.


row_count,tablename
0,demo.amzn_reviews
0,demo.amzn_reviews_partitioned


### 6.2. Familiarize yourself with the Partitioned Table Design and Present Basic Demographics

#### 6.2.1. Retrieve Partitioned Table Design

In [4]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-2-1-partition-table-design.sql

display_html('\n'.join(sqlfilecode), raw=True)

query = !cat script/6-2-1-partition-table-design.sql

%sql $DB_USER@$DB_SERVER {''.join(query)}

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT 
 partitionboundary , 
 partitiontablename , 
 partitionname , 
 partitionlevel , 
 partitionrank 
 FROM pg_partitions 
 WHERE tablename = 'amzn_reviews_partitioned' ;

21 rows affected.


partitionboundary,partitiontablename,partitionname,partitionlevel,partitionrank
PARTITION year1995 START ('1995-01-01'::date) END ('1996-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year1995,year1995,0,1
PARTITION year1996 START ('1996-01-01'::date) END ('1997-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year1996,year1996,0,2
PARTITION year1997 START ('1997-01-01'::date) END ('1998-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year1997,year1997,0,3
PARTITION year1998 START ('1998-01-01'::date) END ('1999-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year1998,year1998,0,4
PARTITION year1999 START ('1999-01-01'::date) END ('2000-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year1999,year1999,0,5
PARTITION year2000 START ('2000-01-01'::date) END ('2001-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year2000,year2000,0,6
PARTITION year2001 START ('2001-01-01'::date) END ('2002-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year2001,year2001,0,7
PARTITION year2002 START ('2002-01-01'::date) END ('2003-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year2002,year2002,0,8
PARTITION year2003 START ('2003-01-01'::date) END ('2004-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year2003,year2003,0,9
PARTITION year2004 START ('2004-01-01'::date) END ('2005-01-01'::date) EVERY ('1 year'::interval),amzn_reviews_partitioned_1_prt_year2004,year2004,0,10


#### 6.2.2. Row Count per Partition

In [5]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-2-2-row_count_per_partition.sql

display_html('\n'.join(sqlfilecode), raw=True)

query = !cat script/6-2-2-row_count_per_partition.sql

%sql $DB_USER@$DB_SERVER {''.join(query)}

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT 
 tableoid :: regclass AS partition_name , 
 count ( * ) AS row_count 
 FROM 
 demo . amzn_reviews_partitioned 
 GROUP BY 1 
 ORDER BY 1 ;

21 rows affected.


partition_name,row_count
demo.amzn_reviews_partitioned_1_prt_year1995,201
demo.amzn_reviews_partitioned_1_prt_year1996,4689
demo.amzn_reviews_partitioned_1_prt_year1997,36846
demo.amzn_reviews_partitioned_1_prt_year1998,162781
demo.amzn_reviews_partitioned_1_prt_year1999,435009
demo.amzn_reviews_partitioned_1_prt_year2000,981615
demo.amzn_reviews_partitioned_1_prt_year2001,880854
demo.amzn_reviews_partitioned_1_prt_year2002,919312
demo.amzn_reviews_partitioned_1_prt_year2003,1023003
demo.amzn_reviews_partitioned_1_prt_year2004,1066129


#### 6.2.3. Row Count per Partition & Segment

In [6]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-2-3-row-count-per-partition-segment.sql

display_html('\n'.join(sqlfilecode), raw=True)

query = !cat script/6-2-3-row-count-per-partition-segment.sql

%sql $DB_USER@$DB_SERVER {''.join(query)}

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT 
 tableoid :: regclass as partition_name , 
 gp_segment_id as segment_id , 
 count ( * ) AS segment_count , 
 SUM ( COUNT ( * )) OVER ( PARTITION BY tableoid :: regclass ) AS partition_count 
 FROM 
 demo . amzn_reviews_partitioned 
 GROUP BY 1 , 2 
 ORDER BY 1 , 2 ;

1007 rows affected.


partition_name,segment_id,segment_count,partition_count
demo.amzn_reviews_partitioned_1_prt_year1995,0,8,201
demo.amzn_reviews_partitioned_1_prt_year1995,1,4,201
demo.amzn_reviews_partitioned_1_prt_year1995,2,2,201
demo.amzn_reviews_partitioned_1_prt_year1995,3,4,201
demo.amzn_reviews_partitioned_1_prt_year1995,4,4,201
demo.amzn_reviews_partitioned_1_prt_year1995,5,5,201
demo.amzn_reviews_partitioned_1_prt_year1995,6,9,201
demo.amzn_reviews_partitioned_1_prt_year1995,7,5,201
demo.amzn_reviews_partitioned_1_prt_year1995,8,3,201
demo.amzn_reviews_partitioned_1_prt_year1995,9,6,201


### 6.3. Partitioned Table Size and Disk Space Usage

After you create the partitioned table structure, top-level parent tables are empty. Data is routed to the bottom-level child table partitions. In a multi-level partition design, only the subpartitions at the bottom of the hierarchy can contain data.

Compare the output below with the [Non-Partitioned Table Size and Disk Usage](http://127.0.0.1:9900/notebooks/gp-demo/AWS-GP-demo-2.ipynb#4.5.2.-Using-the-gp_toolkit-Administrative-Schema-(Greenplum-5.x)).

When you use `COPY` or `INSERT` to load data into a parent table, the data is automatically rerouted to the correct partition, just like a regular table.

In [7]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-3-partitioned-table-size-disk.sql

display_html('\n'.join(sqlfilecode), raw=True)

query = !cat script/6-3-partitioned-table-size-disk.sql

%sql $DB_USER@$DB_SERVER {''.join(query)}

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 SELECT sotd . schemaname || '.' || sotd . tablename as tablename , 
 tabledisksize , 
 uncompressedsize , 
 tablesize , 
 indexsize , 
 toastsize , 
 othersize , 
 sopaid . partitionschemaname || '.' || sopaid . partitiontablename as partitionname , 
 sopaid . partitiontablesize , 
 sopaid . partitionindexsize 
 FROM ( 
 SELECT sotd . sotdoid as oid , 
 sotd . sotdschemaname as schemaname , 
 sotd . sotdtablename as tablename , 
 pg_size_pretty ( sotd . sotdsize :: BIGINT ) as tablesize , 
 pg_size_pretty ( sotd . sotdtoastsize :: BIGINT ) as toastsize , 
 pg_size_pretty ( sotd . sotdadditionalsize :: BIGINT ) as othersize 
 FROM 
 gp_toolkit . gp_size_of_table_disk as sotd 
 WHERE 
 sotd . sotdschemaname || '.' || sotd . sotdtablename = 'demo.amzn_reviews_partitioned' ) sotd 
 INNER JOIN ( 
 SELECT sotaid . sotaidoid as oid , 
 sotaid . sotaidschemaname as schemaname , 
 sotaid . sotaidtablename as tablename , 
 pg_size_pretty ( sotaid . sotaidtablesize :: BIGINT ) as tabledisksize , 
 pg_size_pretty ( sotaid . sotaididxsize :: BIGINT ) as indexsize 
 FROM 
 gp_toolkit . gp_size_of_table_and_indexes_disk as sotaid 
 WHERE 
 sotaid . sotaidschemaname || '.' || sotaid . sotaidtablename = 'demo.amzn_reviews_partitioned' ) sotaid 
 ON sotd . oid = sotaid . oid 
 INNER JOIN ( 
 SELECT sotu . sotuoid as oid , 
 sotu . sotuschemaname as schemaname , 
 sotu . sotutablename as tablename , 
 pg_size_pretty ( sotu . sotusize :: BIGINT ) as uncompressedsize 
 FROM 
 gp_toolkit . gp_size_of_table_uncompressed as sotu 
 WHERE 
 sotu . sotuschemaname || '.' || sotu . sotutablename = 'demo.amzn_reviews_partitioned' ) sotu 
 ON sotaid . oid = sotu . oid 
 RIGHT OUTER JOIN ( 
 SELECT 
 sopaidparentoid as parenttableoid , 
 pg_size_pretty ( sopaidpartitiontablesize :: BIGINT ) as partitiontablesize , 
 pg_size_pretty ( sopaidpartitionindexessize :: BIGINT ) as partitionindexsize , 
 sopaidpartitionschemaname as partitionschemaname , 
 sopaidpartitiontablename as partitiontablename 
 FROM 
 gp_toolkit . gp_size_of_partition_and_indexes_disk 
 WHERE 
 sopaidparentschemaname || '.' || sopaidparenttablename = 'demo.amzn_reviews_partitioned' ) sopaid 
 ON sopaid . parenttableoid = sotd . oid ;

21 rows affected.


tablename,tabledisksize,uncompressedsize,tablesize,indexsize,toastsize,othersize,partitionname,partitiontablesize,partitionindexsize
demo.amzn_reviews_partitioned,1568 kB,1568 kB,0 bytes,0 bytes,1568 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year1995,3072 kB,0 bytes
demo.amzn_reviews_partitioned,1568 kB,1568 kB,0 bytes,0 bytes,1568 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year1996,6368 kB,0 bytes
demo.amzn_reviews_partitioned,1568 kB,1568 kB,0 bytes,0 bytes,1568 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year1997,33 MB,0 bytes
demo.amzn_reviews_partitioned,1568 kB,1568 kB,0 bytes,0 bytes,1568 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year1998,121 MB,0 bytes
demo.amzn_reviews_partitioned,1568 kB,1568 kB,0 bytes,0 bytes,1568 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year1999,308 MB,0 bytes
demo.amzn_reviews_partitioned,1568 kB,1568 kB,0 bytes,0 bytes,1568 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year2000,860 MB,0 bytes
demo.amzn_reviews_partitioned,1568 kB,1568 kB,0 bytes,0 bytes,1568 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year2001,870 MB,0 bytes
demo.amzn_reviews_partitioned,1568 kB,1568 kB,0 bytes,0 bytes,1568 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year2002,934 MB,0 bytes
demo.amzn_reviews_partitioned,1568 kB,1568 kB,0 bytes,0 bytes,1568 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year2003,1067 MB,0 bytes
demo.amzn_reviews_partitioned,1568 kB,1568 kB,0 bytes,0 bytes,1568 kB,0 bytes,demo.amzn_reviews_partitioned_1_prt_year2004,1192 MB,0 bytes


### 6.4. Verify your Partition Strategy and Demonstrate *Partition Elimination* functionality

When a table is partitioned based on the query predicate, you can use `EXPLAIN` to verify that the query optimizer scans only the relevant data to examine the query plan. For example, the `demo.amzn_reviews_2` table is date-range partitioned by year. 

### Example 2: `SELECT` data for a single day (`2011-07-12`):

The query plan for this query should show a table scan of only the following tables:

- the default partition returning 0-1 rows (if your partition design has one)
- the 2011 partition (`	demo.amzn_reviews_2_1_prt_year2011`) returning ***some number*** of rows

To confirm, execute the `EXPLAIN` query and check the query plan:

In [15]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-4-explain-example-1.sql

display_html('\n'.join(sqlfilecode), raw=True)

explain_output = !cat script/6-4-explain-example-1.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 

display_html('\n'.join(explain_output), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN SELECT * FROM demo . amzn_reviews_partitioned WHERE review_date = '2011-07-12' ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 QUERY PLAN 
 ----------------------------------------------------------------------------------------------------------------------- 
 Gather Motion 48 : 1 ( slice1 ; segments : 48 ) ( cost = 0.00..431.00 rows = 1 width = 390 ) 
 -> Sequence ( cost = 0.00..431.00 rows = 1 width = 390 ) 
 -> Partition Selector for amzn_reviews_partitioned ( dynamic scan id : 1 ) ( cost = 10.00..100.00 rows = 3 width = 4 ) 
 Partitions selected : 1 ( out of 21 ) 
 -> Dynamic Seq Scan on amzn_reviews_partitioned ( dynamic scan id : 1 ) ( cost = 0.00..431.00 rows = 1 width = 390 ) 
 Filter : ( review_date = '2011-07-12' :: date ) 
 Optimizer : Pivotal Optimizer ( GPORCA ) 
 ( 7 rows )

### Example 3 : "Single-Partition" `SELECT`

### Calculate MTD (Month-to-date) Number of Reviews for the dates, up to '25 March 2015' - Original/Non-Partitioned Table (`demo.amzn_reviews`)

In [16]:
query = !cat script/6-4-calendar-foundation.sql
%sql $DB_USER@$DB_SERVER {''.join(query)}

Done.
Done.
7374 rows affected.
Done.
Done.
7374 rows affected.
Done.
Done.


[]

In [17]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-4-explain-example-3-1.sql

display_html('\n'.join(sqlfilecode), raw=True)

explain_output = !cat script/6-4-explain-example-3-1.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 

display_html('\n'.join(explain_output), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT COUNT ( * ), 
 cdate_mtd 
 FROM demo . amzn_reviews , 
 demo . calendar_mtd 
 WHERE 
 amzn_reviews . review_date = calendar_mtd . cdate_mtd 
 AND cdate = '2015-03-25' :: DATE 
 GROUP BY cdate_mtd 
 ORDER BY cdate_mtd ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 QUERY PLAN 
 ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- 
 Gather Motion 48 : 1 ( slice4 ; segments : 48 ) ( cost = 0.00..2594.85 rows = 31 width = 12 ) ( actual time = 2265.864..2265.870 rows = 25 loops = 1 ) 
 Merge Key : calendar . cdate 
 -> Sort ( cost = 0.00..2594.85 rows = 1 width = 12 ) ( actual time = 2265.286..2265.288 rows = 2 loops = 1 ) 
 Sort Key : calendar . cdate 
 Sort Method : quicksort Memory : 1584 kB 
 -> GroupAggregate ( cost = 0.00..2594.85 rows = 1 width = 12 ) ( actual time = 2265.254..2265.263 rows = 2 loops = 1 ) 
 Group Key : calendar . cdate 
 -> Sort ( cost = 0.00..2594.85 rows = 1 width = 12 ) ( actual time = 2265.239..2265.244 rows = 96 loops = 1 ) 
 Sort Key : calendar . cdate 
 Sort Method : quicksort Memory : 1584 kB 
 -> Redistribute Motion 48 : 48 ( slice3 ; segments : 48 ) ( cost = 0.00..2594.85 rows = 1 width = 12 ) ( actual time = 1225.682..2265.157 rows = 96 loops = 1 ) 
 Hash Key : calendar . cdate 
 -> Result ( cost = 0.00..2594.85 rows = 1 width = 12 ) ( actual time = 1263.460..1263.487 rows = 25 loops = 1 ) 
 -> HashAggregate ( cost = 0.00..2594.85 rows = 1 width = 12 ) ( actual time = 1263.459..1263.480 rows = 25 loops = 1 ) 
 Group Key : calendar . cdate 
 Extra Text : ( seg0 ) Hash chain length 1.2 avg , 2 max , using 20 of 32 buckets ; total 0 expansions . 
 
 -> Hash Join ( cost = 0.00..2593.23 rows = 12953 width = 4 ) ( actual time = 43.359..1348.433 rows = 97243 loops = 1 ) 
 Hash Cond : ( amzn_reviews . review_date = calendar . cdate ) 
 Extra Text : ( seg6 ) Hash chain length 1.0 avg , 1 max , using 25 of 4194304 buckets . 
 -> Seq Scan on amzn_reviews ( cost = 0.00..1152.85 rows = 3147377 width = 4 ) ( actual time = 0.031..927.619 rows = 3150425 loops = 1 ) 
 -> Hash ( cost = 862.10..862.10 rows = 31 width = 4 ) ( actual time = 22.481..22.481 rows = 25 loops = 1 ) 
 -> Broadcast Motion 48 : 48 ( slice2 ; segments : 48 ) ( cost = 0.00..862.10 rows = 31 width = 4 ) ( actual time = 7.448..22.459 rows = 25 loops = 1 ) 
 -> Hash Join ( cost = 0.00..862.10 rows = 1 width = 4 ) ( actual time = 28.032..38.841 rows = 2 loops = 1 ) 
 Hash Cond : (( calendar . from_mtd = calendar_1 . from_mtd ) AND ( calendar . from_ytd = calendar_1 . from_ytd )) 
 Join Filter : ( calendar_1 . cdate >= calendar . cdate ) 
 Extra Text : ( seg5 ) Hash chain length 1.0 avg , 1 max , using 1 of 4194304 buckets . 
 -> Seq Scan on calendar ( cost = 0.00..431.00 rows = 154 width = 12 ) ( actual time = 0.013..0.024 rows = 182 loops = 1 ) 
 -> Hash ( cost = 431.01..431.01 rows = 1 width = 12 ) ( actual time = 0.032..0.032 rows = 1 loops = 1 ) 
 -> Broadcast Motion 48 : 48 ( slice1 ; segments : 48 ) ( cost = 0.00..431.01 rows = 1 width = 12 ) ( actual time = 0.016..0.026 rows = 1 loops = 1 ) 
 -> Seq Scan on calendar calendar_1 ( cost = 0.00..431.01 rows = 1 width = 12 ) ( actual time = 0.056..0.057 rows = 1 loops = 1 ) 
 Filter : ( cdate = '2015-03-25' :: date ) 
 Planning time : 112.952 ms 
 ( slice0 ) Executor memory : 279 K bytes . 
 ( slice1 ) Executor memory : 60 K bytes avg x 48 workers , 60 K bytes max ( seg0 ) . 
 ( slice2 ) Executor memory : 32858 K bytes avg x 48 workers , 32868 K bytes max ( seg1 ) . Work_mem : 1 K bytes max . 
 ( slice3 ) Executor memory : 32940 K bytes avg x 48 workers , 32940 K bytes max ( seg0 ) . Work_mem : 1 K bytes max . 
 ( slice4 ) Executor memory : 132 K bytes avg x 48 workers , 136 K bytes max ( seg5 ) . Work_mem : 65 K bytes max . 
 Memory used : 3223552 kB 
 Optimizer : Pivotal Optimizer ( GPORCA ) 
 Execution time : 2283.444 ms 
 ( 41 rows )

### Calculate MTD (Month-to-date) Number of Reviews for the dates, up to '25 March 2015' - Partitioned Table (`demo.amzn_reviews_partitioned`)

In [18]:
sqlfilecode = !pygmentize -f html -O full,style=colorful -l postgres script/6-4-explain-example-3-2.sql

display_html('\n'.join(sqlfilecode), raw=True)

explain_output= !cat script/6-4-explain-example-3-2.sql \
    | psql $CONNECTION_STRING | pygmentize -f html -O full,style=colorful -l psql 

display_html('\n'.join(explain_output), raw=True)

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 EXPLAIN ANALYZE 
 SELECT COUNT ( * ), 
 cdate_mtd 
 FROM demo . amzn_reviews_partitioned , 
 demo . calendar_mtd 
 WHERE 
 amzn_reviews_partitioned . review_date = calendar_mtd . cdate_mtd 
 AND cdate = '2015-03-25' :: DATE 
 GROUP BY cdate_mtd 
 ORDER BY cdate_mtd ;

<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN"
 "http://www.w3.org/TR/html4/strict.dtd">
<!--
generated by Pygments 
Copyright 2006-2019 by the Pygments team.
Licensed under the BSD license, see LICENSE for details.
-->
 
 
 
 
 
 
 
 

 QUERY PLAN 
 ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- 
 Gather Motion 48 : 1 ( slice4 ; segments : 48 ) ( cost = 0.00..1300.27 rows = 31 width = 12 ) ( actual time = 664.294..664.313 rows = 25 loops = 1 ) 
 Merge Key : calendar . cdate 
 -> Sort ( cost = 0.00..1300.27 rows = 1 width = 12 ) ( actual time = 663.917..663.918 rows = 2 loops = 1 ) 
 Sort Key : calendar . cdate 
 Sort Method : quicksort Memory : 1584 kB 
 -> GroupAggregate ( cost = 0.00..1300.27 rows = 1 width = 12 ) ( actual time = 663.901..663.909 rows = 2 loops = 1 ) 
 Group Key : calendar . cdate 
 -> Sort ( cost = 0.00..1300.27 rows = 1 width = 12 ) ( actual time = 663.889..663.892 rows = 96 loops = 1 ) 
 Sort Key : calendar . cdate 
 Sort Method : quicksort Memory : 1584 kB 
 -> Redistribute Motion 48 : 48 ( slice3 ; segments : 48 ) ( cost = 0.00..1300.27 rows = 1 width = 12 ) ( actual time = 364.083..663.809 rows = 96 loops = 1 ) 
 Hash Key : calendar . cdate 
 -> Result ( cost = 0.00..1300.27 rows = 1 width = 12 ) ( actual time = 369.992..370.007 rows = 25 loops = 1 ) 
 -> HashAggregate ( cost = 0.00..1300.27 rows = 1 width = 12 ) ( actual time = 369.990..370.001 rows = 25 loops = 1 ) 
 Group Key : calendar . cdate 
 Extra Text : ( seg0 ) Hash chain length 1.2 avg , 2 max , using 20 of 32 buckets ; total 0 expansions . 
 
 -> Hash Join ( cost = 0.00..1298.65 rows = 12946 width = 4 ) ( actual time = 41.497..361.542 rows = 97243 loops = 1 ) 
 Hash Cond : ( amzn_reviews_partitioned . review_date = calendar . cdate ) 
 Extra Text : ( seg6 ) Hash chain length 1.0 avg , 1 max , using 25 of 4194304 buckets . 
 -> Dynamic Seq Scan on amzn_reviews_partitioned ( dynamic scan id : 1 ) ( cost = 0.00..433.99 rows = 12946 width = 4 ) ( actual time = 0.132..231.013 rows = 875324 loops = 1 ) 
 Partitions scanned : Avg 1.0 ( out of 21 ) x 48 workers . Max 1 parts ( seg0 ) . 
 -> Hash ( cost = 100.00..100.00 rows = 3 width = 4 ) ( actual time = 13.484..13.484 rows = 25 loops = 1 ) 
 -> Partition Selector for amzn_reviews_partitioned ( dynamic scan id : 1 ) ( cost = 10.00..100.00 rows = 3 width = 4 ) ( actual time = 0.416..13.451 rows = 25 loops = 1 ) 
 -> Broadcast Motion 48 : 48 ( slice2 ; segments : 48 ) ( cost = 0.00..862.10 rows = 31 width = 4 ) ( actual time = 0.010..12.980 rows = 25 loops = 1 ) 
 -> Hash Join ( cost = 0.00..862.10 rows = 1 width = 4 ) ( actual time = 27.237..40.677 rows = 2 loops = 1 ) 
 Hash Cond : (( calendar . from_mtd = calendar_1 . from_mtd ) AND ( calendar . from_ytd = calendar_1 . from_ytd )) 
 Join Filter : ( calendar_1 . cdate >= calendar . cdate ) 
 Extra Text : ( seg5 ) Hash chain length 1.0 avg , 1 max , using 1 of 4194304 buckets . 
 -> Seq Scan on calendar ( cost = 0.00..431.00 rows = 154 width = 12 ) ( actual time = 0.012..0.029 rows = 182 loops = 1 ) 
 -> Hash ( cost = 431.01..431.01 rows = 1 width = 12 ) ( actual time = 0.051..0.051 rows = 1 loops = 1 ) 
 -> Broadcast Motion 48 : 48 ( slice1 ; segments : 48 ) ( cost = 0.00..431.01 rows = 1 width = 12 ) ( actual time = 0.029..0.047 rows = 1 loops = 1 ) 
 -> Seq Scan on calendar calendar_1 ( cost = 0.00..431.01 rows = 1 width = 12 ) ( actual time = 0.021..0.022 rows = 1 loops = 1 ) 
 Filter : ( cdate = '2015-03-25' :: date ) 
 Planning time : 149.644 ms 
 ( slice0 ) Executor memory : 288 K bytes . 
 ( slice1 ) Executor memory : 44 K bytes avg x 48 workers , 44 K bytes max ( seg0 ) . 
 ( slice2 ) Executor memory : 32858 K bytes avg x 48 workers , 32868 K bytes max ( seg1 ) . Work_mem : 1 K bytes max . 
 ( slice3 ) Executor memory : 32972 K bytes avg x 48 workers , 32972 K bytes max ( seg0 ) 

## Continue to Part 4 of Greenplum Database  Concepts Explained, **[Table Storage Models](AWS-GP-demo-4.ipynb)**.